In [1]:
import open3d as o3d
import numpy as np
import math, pandas, pickle
import scipy
from scipy import sparse
from sklearn.neighbors import KDTree
from sklearn.metrics.pairwise import euclidean_distances
import gudhi as gd
from gudhi.weighted_rips_complex import WeightedRipsComplex
from gudhi.dtm_rips_complex import DTMRipsComplex
from ripser import ripser
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from gudhi.tensorflow import RipsLayer
import plotly.graph_objects as go
import ot.plot
import matplotlib.pylab as pl
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from utilyze import *


In [3]:
def find_align(source_pc, target_pc, output_icp = False, max_correspondence_distance=.1):
    source_cloud = o3d.geometry.PointCloud()
    source_cloud.points = o3d.utility.Vector3dVector(source_pc)
    target_cloud = o3d.geometry.PointCloud()
    target_cloud.points = o3d.utility.Vector3dVector(target_pc)
    # Perform ICP
    icp_result = o3d.pipelines.registration.registration_icp(
        source_cloud,
        target_cloud,
        max_correspondence_distance=max_correspondence_distance,
        estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    )
    # Get the aligned source cloud
    aligned_source_cloud = source_cloud.transform(icp_result.transformation)
    pc1_regis = np.asarray(aligned_source_cloud.points)
    if output_icp:
        output = (pc1_regis, icp_result)
    else:
        output = pc1_regis
    return output

In [2]:
random_seed()
# Raw data is downloaded from '# https://gitlab.com/alexpieloch/PersistentHomologyAnalysisofBrainArteryTrees/-/tree/master/data/OriginalBrainTreeData'
data_pc = np.load("Data/data_pc.pkl", allow_pickle=True)
print('Original pointcloud size:',data_pc[0].shape)
# Subsample
number_pts = 2000 
data_pc_subsample = []
for i in range(len(data_pc)):
    if i not in [40, 45]:
        pc = data_pc[i]
        all_idx = np.random.permutation(pc.shape[0])
        for j in range(1):
            small_idx = all_idx[int(j * number_pts) : int((j+1) * number_pts)]
            pc_sub = pc[small_idx]
            # normalize
            centered_pc_sub = np.copy(pc_sub)
            centered_pc_sub[:,0] -= np.mean(pc_sub[:,0])
            centered_pc_sub[:,1] -= np.mean(pc_sub[:,1])
            centered_pc_sub[:,2] -= np.mean(pc_sub[:,2])
            data_pc_subsample.append(centered_pc_sub)
data_pc = data_pc_subsample

# Normalize
mean_max = np.mean([np.max(abs(data_pc[i]),axis=0) for i in range(len(data_pc))])
for i in range(len(data_pc)):
    data_pc[i] /= mean_max * 2
print('Randomness check: [88916 11866 12750 14216 86734]',  all_idx[:5])
print('Number of data: ',len(data_pc))
print('Sampled pc size: ',data_pc[0].shape)

Original pointcloud size: (116361, 3)
Randomness check: [88916 11866 12750 14216 86734] [88916 11866 12750 14216 86734]
Number of data:  96
Sampled pc size:  (2000, 3)


In [ ]:
# compute the pc-wise Was_icp distance -> interate every i, find the smallest, compute the gradient, subsample 500 pts and gradvec correspondingly
n = len(data_pc)
distance_matrix = np.zeros((n, n))
# Compute the pairwise Wasserstein distances between point clouds
for i in range(n):
    start_time = time.time()  # Start measuring the time
    for j in range(i, n):
        pc1,pc2 = data_pc[i],data_pc[j]
        pc1_regis = find_align(pc1, pc2)
        # Compute wass
        M = ot.dist(pc1_regis, pc2)
        # Emprical distribution
        d1,d2 = np.ones((number_pts,)) / number_pts, np.ones((number_pts,)) / number_pts
        # OT matrix
        emd = ot.emd(d1,d2,M, numItermax=1e6) #default numItermax 1e5
        # Was distance
        W = np.sum(emd * M)
        distance_matrix[i][j] = W
        distance_matrix[j][i] = W
    break

# import pickle
# with open("/content/drive/MyDrive/ICG_WAS_DISTMAT.pkl", 'wb') as f:
#     pickle.dump(distance_matrix, f)

In [4]:
# with open('ICG_WAS_DISTMAT.pkl', 'rb') as f:
#     distance_matrix = pickle.load(f)

In [ ]:
# Gradient estimate (finding the neareast data with different lable.)
data_grad = []
data_pc_real = []
number_pts_real = 500

df = pandas.read_csv('Data/label.csv')
gender = np.array(df['gender'])
label = []
for i in range(gender.shape[0]):
    if i not in [48,50,63,66,69,72,78,84,98,101,103]:
        label.append(gender[i])
labels_raw=np.array(label)
labels_raw -= 1
labels = []
for i in range(len(labels_raw)):
    if i not in [40, 45]:
        label_pc = labels_raw[i]
        for j in range(1):
            labels.append(label_pc)
labels = np.array(labels)


for i,pc in enumerate(data_pc):
    # find the most neareast pc with diff label
    label = labels[i]
    index_otherclass = np.where(labels!=label)[0]
    distances_otherclass = np.array([distance_matrix[i, j] for j in index_otherclass])
    j = index_otherclass[np.argmin(distances_otherclass)]
    # find the OT vector
    pc1 = data_pc[i]
    pc2 = data_pc[j]
    pc1_regis = find_align(pc1, pc2)
    M = ot.dist(pc1_regis, pc2)
    n = pc1_regis.shape[0]
    d1,d2 = np.ones((n,)) / n, np.ones((n,)) / n
    emd = ot.emd(d1,d2,M)
    indices = np.argwhere(emd == np.max(emd))
    vec = [pc2[l] - pc1_regis[k] for k, l in indices]
    vec = np.array(vec)
    random_seed()
    all_idx = np.random.permutation(pc1_regis.shape[0])
    for iter in range(3):
        small_idx = all_idx[number_pts_real*iter: number_pts_real*(iter+1)]
        pc_real = pc1_regis[small_idx]
        vec_real = vec[small_idx]
        vec_real = np.array(vec_real).reshape(-1,)
        data_grad.append(vec_real)
        data_pc_real.append(pc_real)
# import pickle
# with open("ICG_WAS_PC_3.pkl", 'wb') as f:
#     pickle.dump(data_pc_real, f)

# with open("ICG_WAS_GRAD_3.pkl", 'wb') as f:
#     pickle.dump(data_grad, f)

In [9]:
with open('ICG_WAS_PC_3.pkl', 'rb') as f:
    data_pc = pickle.load(f)
data_pc = np.array(data_pc)

with open('ICG_WAS_GRAD_3.pkl', 'rb') as f:
    data_grad = pickle.load(f)

In [ ]:
# compute pull-back norms
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

xrange = [.0, .25]
yrange = [-.05, .2]
maximum_edge_length = .25
gauss_sigmas = np.linspace(1e-5, 1e-4, 8)
number_pixels = [15, 20, 25, 30]

Pbn = {}
Rank = {}
for number_pixel in number_pixels:
    Pbn[number_pixel] = {}
    Rank[number_pixel] = {}
    for gauss_sigma in gauss_sigmas:
        Pbn[number_pixel][gauss_sigma] = []
        Rank[number_pixel][gauss_sigma] = []


random_seed()
minisize = len(data_pc)
idx = np.random.permutation(len(data_pc))[:minisize]

for number_pixel in number_pixels:
    for gauss_sigma in gauss_sigmas:
        random_seed()
        weightfunc = Weight(method='linear', b=yrange[1])
        jacobians, pis, ranks = Collect_jacobian_and_rank_and_pi(np.array(data_pc)[idx], get_jacobian_rips, xrange, yrange,
                                                number_pixel=number_pixel, gauss_sigma=gauss_sigma, 
                                                weight_func = weightfunc, max_edge_length=maximum_edge_length, 
                                                homology_dimensions=[1], normalize=False, normalize_jac=True)
        
        grads = np.array(data_grad)[idx]
        assert len(grads)==len(jacobians)
        Pbn[number_pixel][gauss_sigma] = [np.linalg.norm(jacobians[k] @ grads[k]) for k in range(len(grads))]
        Rank[number_pixel][gauss_sigma] = ranks
        #print(number_pixel)

# import pickle
# with open("Pbn_resvar.pkl", "wb") as file:
#     pickle.dump(Pbn, file)

# with open("Rank_resvar.pkl", "wb") as file:
#     pickle.dump(Rank, file)

In [14]:
# downstream performance
df = pandas.read_csv('Data/label.csv')
gender = np.array(df['gender'])
label = []
for i in range(gender.shape[0]):
    if i not in [48,50,63,66,69,72,78,84,98,101,103]:
        label.append(gender[i])
labels_raw=np.array(label)
#print(np.where(labels_raw==3))
#print(np.where(labels_raw==4))
#print(len(labels_raw))
labels_raw -= 1

labels = []
for i in range(len(labels_raw)):
    if i not in [40, 45]:
        label_pc = labels_raw[i]
        for j in range(3):
            labels.append(label_pc)        
labels = np.array(labels)
#print(len(labels))

In [12]:
from sklearn.metrics import log_loss


def train_lr(train_data, train_labels, val_data, val_labels, noise_level = .01, C=1):
    random_seed()
    lr = LogisticRegression(max_iter=300, C=C) #solver
    lr.fit(train_data, train_labels)
    test_accuracy = accuracy_score(val_labels, lr.predict(val_data))
    test_loss = log_loss(val_labels, lr.predict_proba(val_data))
    train_accuracy = accuracy_score(train_labels, lr.predict(train_data))
    train_loss = log_loss(train_labels, lr.predict_proba(train_data))

    #robust test
    val_data_noisy = val_data + np.random.normal(0,noise_level, size = val_data.shape)
    robust_test_loss = log_loss(val_labels, lr.predict_proba(val_data_noisy))
    robust_test_accuracy = accuracy_score(val_labels, lr.predict(val_data_noisy))

    return train_loss, train_accuracy, test_loss, test_accuracy, robust_test_loss, robust_test_accuracy

def train_cnn(train_data, train_labels, val_data, val_labels, noise_level=.01, epochs=10, batch_size=32):
    random_seed()
    # Define the CNN model
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(train_data.shape[1], train_data.shape[2], 1)),
        tf.keras.layers.MaxPooling2D((2, 2)), 
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    # Compile the model
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    # Train the model
    history = model.fit(train_data, train_labels,
                        validation_data=(val_data, val_labels),
                        epochs=epochs,
                        batch_size=batch_size,
                        verbose=0)
    # Evaluate the model on test data
    train_loss, train_accuracy = model.evaluate(train_data, train_labels, verbose=0)
    test_loss, test_accuracy = model.evaluate(val_data, val_labels, verbose=0)
    # Robust test
    val_data_noisy = val_data + np.random.normal(0,noise_level, size = val_data.shape)
    robust_test_loss, robust_test_accuracy = model.evaluate(val_data_noisy, val_labels, verbose=0)
    return train_loss, train_accuracy, test_loss, test_accuracy, robust_test_loss, robust_test_accuracy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

xrange = [.0, .25]
yrange = [-.05, .2]
maximum_edge_length = .25


gauss_sigmas = np.linspace(1e-5, 1e-4, 8)
number_pixels = [15, 20, 25, 30]

random_seed()
Testacc = {}
Robustacc = {}
Testacc_cnn = {}
Robustacc_cnn = {}

for number_pixel in number_pixels:
    Testacc[number_pixel] = {}
    Robustacc[number_pixel] = {}
    Testacc_cnn[number_pixel] = {}
    Robustacc_cnn[number_pixel] = {}
    for gauss_sigma in gauss_sigmas:
        Testacc[number_pixel][gauss_sigma] = []
        Robustacc[number_pixel][gauss_sigma] = []
        Testacc_cnn[number_pixel][gauss_sigma] = []
        Robustacc_cnn[number_pixel][gauss_sigma] = []


from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=7)

for number_pixel in number_pixels:
    for gauss_sigma in gauss_sigmas:
        random_seed()
        weightfunc = Weight(method='linear', b=yrange[1])
        data_pi = Collect_pi(data_pc, xrange, yrange, number_pixel=number_pixel, gauss_sigma=gauss_sigma, weight_func = weightfunc, max_edge_length=maximum_edge_length, homology_dimensions=[1], normalize=False)            
        data_pi /= np.max(data_pi) # normalize feature <-> normal jac
        
        #Logistic
        for train_index, test_index in kf.split(data_pi):
            train_pi, val_pi = data_pi[train_index], data_pi[test_index]
            train_labels, val_labels = labels[train_index], labels[test_index]
            train_loss, train_accuracy, test_loss, test_accuracy, robust_test_loss, robust_test_accuracy = train_lr(train_pi, train_labels, val_pi, val_labels, C=1)
            Testacc[number_pixel][gauss_sigma].append(test_accuracy)
            Robustacc[number_pixel][gauss_sigma].append(robust_test_accuracy)

        #CNN
        data_image = data_pi.reshape(len(data_pi), number_pixel,number_pixel)
        for train_index, test_index in kf.split(data_image):
            train_pi, val_pi = data_image[train_index], data_image[test_index]
            train_labels, val_labels = labels[train_index], labels[test_index]
            train_loss, train_accuracy, test_loss, test_accuracy, robust_test_loss, robust_test_accuracy = train_cnn(train_pi, train_labels, val_pi, val_labels)
            Testacc_cnn[number_pixel][gauss_sigma].append(test_accuracy)
            Robustacc_cnn[number_pixel][gauss_sigma].append(robust_test_accuracy)
        
        #print(number_pixel)

# import pickle

# with open("Testacc_resvar.pkl", "wb") as file:
#     pickle.dump(Testacc, file)

# with open("Robustacc_resvar.pkl", "wb") as file:
#     pickle.dump(Robustacc, file)

# with open("Testacc_resvar_cnn.pkl", "wb") as file:
#     pickle.dump(Testacc_cnn, file)

# with open("Robustacc_resvar_cnn.pkl", "wb") as file:
#     pickle.dump(Robustacc_cnn, file)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
dictionaries = [Pbn, Rank]
titles = ['Pullback norm','Rank']
cmaps = ['Blues','Greys']
gauss_sigmas = np.linspace(1e-5, 1e-4, 8)
number_pixels = [15, 20, 25, 30]
rows = 2
cols = 2
fig, axes = plt.subplots(rows, cols, figsize=(14, 9))
fmts = ['.3f','.1f']
gauss_sigmas_label = np.round(gauss_sigmas * 1e5 * 1e2)/1e2
FS = 14
dictionary = Rank
mean = [[np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
mean = np.flip(np.array(mean), axis=0)
sd = [[np.std(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
sd = np.flip(np.array(sd), axis=0)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["#ffffff","#252525"])
ax = sns.heatmap(mean, annot=False, cmap=cmap, linewidths=.1, ax=axes[0,0],square=True, cbar=False)

ax.set_yticklabels(np.flip(number_pixels, axis=0))
ax.set_xticklabels(gauss_sigmas_label)
ax.set_title("Rank of Jacobian mapping", fontsize=FS)
ax.set_ylabel('Resolution', fontsize=FS)
ax.set_xlabel('Variance in Gaussian Kernel (x$10^{-5}$)', fontsize=FS)

for i in range(mean.shape[0]):
    for j in range(mean.shape[1]):
        value = mean[i, j]
        std_error = sd[i, j]
        annotation_text = "{:.2f}\n±{:.2f}".format(value, std_error)
        pixel_color = mean[i][j]
        annotation_color = 'black' if pixel_color < 1.1 * np.mean(mean) else 'white'
        ax.text(j + 0.5, i + 0.5, annotation_text, ha='center', va='center', color=annotation_color, fontsize=10)
dictionary = Pbn
mean = [[np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
mean = np.flip(np.array(mean), axis=0)
sd = [[np.std(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
sd = np.flip(np.array(sd), axis=0)
ax = sns.heatmap(mean, annot=False, cmap='Blues', linewidths=.1, ax=axes[0,1],square=True, cbar=False, vmin=.085, vmax=.112)
ax.set_yticklabels(np.flip(number_pixels, axis=0))
ax.set_xticklabels(gauss_sigmas_label)
ax.set_title("Pull-back norm of gradient ($ x 10^{-2}$)", fontsize=FS)
ax.set_ylabel('Resolution', fontsize=FS)
ax.set_xlabel('Variance of Gaussian Kernel (x$10^{-5}$)', fontsize=FS)
for i in range(mean.shape[0]):
    for j in range(mean.shape[1]):
        value = mean[i, j]*100
        std_error = sd[i, j]*100
        annotation_text = "{:.2f}\n±{:.2f}".format(value, std_error)
        pixel_color = mean[i][j]
        annotation_color = 'black' if pixel_color < 1 * np.mean(mean) else 'white'
        ax.text(j + 0.5, i + 0.5, annotation_text, ha='center', va='center', color=annotation_color, fontsize=10)
###
dictionary = Testacc
mean = [[ np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
mean = np.flip(np.array(mean), axis=0)
sd = [[ np.std(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
sd = np.flip(np.array(sd), axis=0)
ax = sns.heatmap(mean, annot=False, cmap='Greens', linewidths=.1, ax=axes[1,0], square=True, cbar=False, vmin=.62, vmax=.7)
ax.set_yticklabels(np.flip(number_pixels, axis=0))
ax.set_title('Validation accuracy (%)', fontsize=FS)
ax.set_xticklabels(gauss_sigmas_label)
ax.set_ylabel('Resolution', fontsize=FS)
ax.set_xlabel('Variance of Gaussian Kernel (x$10^{-5}$)', fontsize=FS)
for i in range(mean.shape[0]):
    for j in range(mean.shape[1]):
        value = mean[i, j] *100
        std_error = sd[i, j] *100
        annotation_text = "{:.2f}\n±{:.2f}".format(value, std_error)
        pixel_color = mean[i][j]
        annotation_color = 'black' if pixel_color < 1 * np.mean(mean) else 'white'
        ax.text(j + 0.5, i + 0.5, annotation_text, ha='center', va='center', color=annotation_color, fontsize=10)
dictionary = Robustacc
mean = [[ np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
mean = np.flip(np.array(mean), axis=0)
sd = [[ np.std(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
sd = np.flip(np.array(sd), axis=0)
ax = sns.heatmap(mean, annot=False, cmap='Greens', linewidths=.1, ax=axes[1,1], square=True, cbar=False, vmin=.59, vmax=.7)
ax.set_yticklabels(np.flip(number_pixels, axis=0))
ax.set_title('Robust validation accuracy (%)', fontsize=FS)
ax.set_xticklabels(gauss_sigmas_label)
ax.set_ylabel('Resolution', fontsize=FS)
ax.set_xlabel('Variance of Gaussian Kernel (x$10^{-5}$)', fontsize=FS)
for i in range(mean.shape[0]):
    for j in range(mean.shape[1]):
        value = mean[i, j] *100
        std_error = sd[i, j] *100
        annotation_text = "{:.2f}\n±{:.2f}".format(value, std_error)
        pixel_color = mean[i][j]
        annotation_color = 'black' if pixel_color < 1.0 * np.mean(mean) else 'white'
        ax.text(j + 0.5, i + 0.5, annotation_text, ha='center', va='center', color=annotation_color, fontsize=10)
plt.tight_layout()
#plt.savefig('resvar.png',dpi=300)

In [ ]:
import pickle
with open('Testacc_resvar.pkl', "rb") as file:
    Testacc = pickle.load(file)
with open('Robustacc_resvar.pkl', "rb") as file:
    Robustacc = pickle.load(file)
with open('Pbn_resvar.pkl', "rb") as file:
    Pbn = pickle.load(file)
with open('Rank_resvar.pkl', "rb") as file:
    Rank = pickle.load(file)

with open('Testacc_resvar_cnn.pkl', "rb") as file:
    Testacc_cnn = pickle.load(file)
with open('Robustacc_resvar_cnn.pkl', "rb") as file:
    Robustacc_cnn = pickle.load(file)

gauss_sigmas = np.linspace(1e-5, 1e-4, 8)
number_pixels = [15, 20, 25, 30]

dictionary = Pbn
mean = [[np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
acc_list = np.array(mean).reshape(-1)
a = acc_list
pbn_list = (a - np.min(a)) / (np.max(a) - np.min(a))

dictionary = Testacc
mean = [[np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
acc_list = np.array(mean).reshape(-1)
a = acc_list
acc_list = (a - np.min(a)) / (np.max(a) - np.min(a))

dictionary = Robustacc
mean = [[np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
rob_acc_list = np.array(mean).reshape(-1)
a = rob_acc_list
rob_acc_list = (a - np.min(a)) / (np.max(a) - np.min(a))

dictionary = Rank
mean = [[np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
rank_list = np.array(mean).reshape(-1)
a = np.array(mean).reshape(-1)
rank_list = (a - np.min(a)) / (np.max(a) - np.min(a))

dictionary = Testacc_cnn
mean = [[np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
a = np.array(mean).reshape(-1)
acc_list_cnn = (a - np.min(a)) / (np.max(a) - np.min(a))

dictionary = Robustacc_cnn
mean = [[np.mean(dictionary[key][sub_key]) for sub_key in gauss_sigmas] for key in number_pixels]
a = np.array(mean).reshape(-1)
rob_acc_list_cnn = (a - np.min(a)) / (np.max(a) - np.min(a))


In [ ]:
# logistic

from scipy.stats import pearsonr
correlation_coeff, p_value = pearsonr(pbn_list, acc_list)
print(correlation_coeff, p_value)

correlation_coeff, p_value = pearsonr(pbn_list, rob_acc_list)
print(correlation_coeff, p_value)

correlation_coeff, p_value = pearsonr(rank_list, acc_list)
print(correlation_coeff, p_value)

correlation_coeff, p_value = pearsonr(rank_list, rob_acc_list)
print(correlation_coeff, p_value)

In [ ]:

from matplotlib.lines import Line2D

fig, axs = plt.subplots(1,4,figsize=(20,5))

sns.kdeplot(x=rank_list, y=acc_list,fill=True,ax=axs[0],color='gray',thresh=.05)
axs[0].set_xlabel('Rank of Jacobian \n (a)',fontsize=15)
axs[0].set_ylabel('Validation accuracy',fontsize=15)
axs[0].set_xlim([-0.3, 1.3])
axs[0].set_ylim([-0.3, 1.3])

props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
textstr = 'PCC: 0.48 \n'+r'P-value: 5.3$\times 10^{-3}$'
axs[0].text(0.52, 0.15, textstr, transform=axs[0].transAxes, fontsize=14, verticalalignment='top', bbox=props)

sns.kdeplot(x=rank_list, y=rob_acc_list,fill=True,ax=axs[1],color='gray',thresh=.05)
axs[1].set_xlabel('Rank of Jacobian \n (b)',fontsize=15)
axs[1].set_ylabel('Robust validation accuracy',fontsize=15)
axs[1].set_xlim([-0.3, 1.3])
axs[1].set_ylim([-0.3, 1.3])

props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
textstr = 'PCC: 0.51 \n'+r'P-value: 3.1$\times 10^{-3}$'
axs[1].text(0.52, 0.15, textstr, transform=axs[1].transAxes, fontsize=14, verticalalignment='top', bbox=props)

sns.kdeplot(x=pbn_list, y=acc_list,fill=True,ax=axs[2], thresh=.05)
axs[2].set_xlabel('Pull-back norm \n (c)',fontsize=15)
axs[2].set_ylabel('Validation accuracy',fontsize=15)
axs[2].set_xlim([-0.3, 1.3])
axs[2].set_ylim([-0.3, 1.3])

props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
textstr = 'PCC: 0.83 \n'+r'P-value: 3.4$\times 10^{-9}$'
axs[2].text(0.52, 0.15, textstr, transform=axs[2].transAxes, fontsize=14, verticalalignment='top', bbox=props)



sns.kdeplot(x=pbn_list, y=rob_acc_list,fill=True,ax=axs[3],thresh=.05)
axs[3].set_xlabel('Pull-back norm \n (d)',fontsize=15)
axs[3].set_ylabel('Robust validation accuracy',fontsize=15)
axs[3].set_xlim([-0.3, 1.3])
axs[3].set_ylim([-0.3, 1.3])

props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
textstr = 'PCC: 0.80 \n'+r'P-value: 2.8$\times 10^{-8}$'
axs[3].text(0.52, 0.15, textstr, transform=axs[3].transAxes, fontsize=14, verticalalignment='top', bbox=props)

plt.tight_layout()
#plt.savefig('correlation.png',dpi=300)

In [ ]:
### CNN ###

correlation_coeff, p_value = pearsonr(pbn_list, acc_list_cnn)
print(correlation_coeff, p_value)

correlation_coeff, p_value = pearsonr(pbn_list, rob_acc_list_cnn)
print(correlation_coeff, p_value)

correlation_coeff, p_value = pearsonr(rank_list, acc_list_cnn)
print(correlation_coeff, p_value)

correlation_coeff, p_value = pearsonr(rank_list, rob_acc_list_cnn)
print(correlation_coeff, p_value)

In [ ]:
fig, axs = plt.subplots(1,4,figsize=(20,5))

sns.kdeplot(x=rank_list, y=acc_list_cnn,fill=True,ax=axs[0],color='gray',thresh=.05)
axs[0].set_xlabel('Rank of Jacobian \n (a)',fontsize=15)
axs[0].set_ylabel('Validation accuracy',fontsize=15)
axs[0].set_xlim([-0.3, 1.3])
axs[0].set_ylim([-0.3, 1.3])
ax = axs[0]
props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
textstr = 'PCC: 0.84 \n'+r'P-value: 2.4$\times 10^{-9}$'
ax.text(0.52, 0.15, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)



sns.kdeplot(x=rank_list, y=rob_acc_list_cnn,fill=True,ax=axs[1],color='gray',thresh=.05)
axs[1].set_xlabel('Rank of Jacobian \n (b)',fontsize=15)
axs[1].set_ylabel('Robust validation accuracy',fontsize=15)
axs[1].set_xlim([-0.3, 1.3])
axs[1].set_ylim([-0.3, 1.3])
ax = axs[1]
props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
textstr = 'PCC: 0.72 \n'+r'P-value: 3.5$\times 10^{-6}$'
ax.text(0.52, 0.15, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)


sns.kdeplot(x=pbn_list, y=acc_list_cnn,fill=True,ax=axs[2], thresh=.05)
axs[2].set_xlabel('Pull-back norm \n (c)',fontsize=15)
axs[2].set_ylabel('Validation accuracy',fontsize=15)
axs[2].set_xlim([-0.3, 1.3])
axs[2].set_ylim([-0.3, 1.3])
ax = axs[2]
props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
textstr = 'PCC: 0.78 \n'+r'P-value: 1.5$\times 10^{-7}$'
ax.text(0.52, 0.15, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)


sns.kdeplot(x=pbn_list, y=rob_acc_list_cnn,fill=True,ax=axs[3],thresh=.05)
axs[3].set_xlabel('Pull-back norm \n (d)',fontsize=15)
axs[3].set_ylabel('Robust validation accuracy',fontsize=15)
axs[3].set_xlim([-0.3, 1.3])
axs[3].set_ylim([-0.3, 1.3])
ax = axs[3]
props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
textstr = 'PCC: 0.72 \n'+r'P-value: 3.4$\times 10^{-6}$'
ax.text(0.52, 0.15, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)

plt.tight_layout()
#plt.savefig('correlation_cnn.png',dpi=300)

